# Shor algorithm

This notebook is part of a set of notebooks on different quantum algorithms. The main source of these notes is Introduction to Quantum Information Science by Vlatko Vedral.

## Theory

Shor's algorithm is a method for factorizing a natural number. It combines the quantum phase estimation method with a classical algorithm for deducing primes.

Need to finish this.


In [138]:
import numpy as np
import itertools

In [139]:
# |0> = [1,0], |1> = [0,1]
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

In [140]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on a tensor products of qubit states.

    Intputs:
    f - tensor product of qubit states, in form [[qubit 1 state], [qubit 2 state], ...]
    """

    qNum = f.shape[0]
    fInv = np.empty([qNum,2],dtype=np.complex_)
    fInv[-1] = H @ f[-1]

    for i in reversed(range(qNum-1)): 
        s = f[i]
        for k in range(i+1,qNum):
            if np.allclose(fInv[k],np.array([0,1])):
                s[1] *= np.exp(-2*np.pi*complex(0,1)/(2**(k+1)))
        s = H @ s
        fInv[i] = s
    
    return fInv


In [141]:
def getDecimalLabel(s): 
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    #print(s)
    sBinary = np.empty(s.shape[0])
    for i,ket in enumerate(s):
        if np.allclose(ket,np.array([1,0])):
            sBinary[i] = 0
        elif np.allclose(ket,np.array([0,1])):
            sBinary[i] = 1
    
    number = sum([b*2**i for i,b in enumerate(np.flip(sBinary))])
    return number

        

In [142]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
invf = np.array([[1,complex(0,1)],[1,-1],[1,1]])/np.sqrt(2) 
print(getDecimalLabel(invQFT(invf)))

2.0


In [143]:
f = [np.array([1,complex(0,1)]),np.array([1,-1])]/np.sqrt(2)
print(getDecimalLabel(invQFT(f))) #should be 1

1.0


In [144]:

def phaseEstimation(psiAcc,U,e): #reversed to tensor product convention used in QFT
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    def pow(n,prev=np.identity(N,dtype=np.complex_)):
        if n == 0:
            return prev
        return pow(n-1,U @ prev)
    

    f = np.zeros([psiAcc,2],dtype=np.complex_)

    for i in range(psiAcc):
        s = np.array([1,0],dtype=np.complex_)
        s = H @ s

        #apply U^2^i
        s[1] *= e @ pow(2**i) @ e
        
        f[i] = s
    
    finvQFT = invQFT(f)
    phase = getDecimalLabel(finvQFT)/2**psiAcc #from relating the phase and result of inv QFT 

    return phase


In [145]:
#test for phase esimation, identity matrix, result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0],dtype=np.complex_)
phaseEstimation(9,U1,e1)

0.0

In [146]:
U2 = np.array([[complex(0,1),0],[0,1]])
e2 = np.array([1,0],dtype=np.complex_)
T = 9
phaseEstimation(T,U2,e2)

0.25

In [147]:
p = 0.5
U3 = np.array([[np.exp(2*np.pi*p*complex(0,1)),0],[0,1]],dtype=np.complex_) #complex(1,1)/np.sqrt(2)
e3 = np.array([1,0],dtype=np.complex_)
T3 = 8
phaseEstimation(T3,U3,e3) #the issue comes from not being able to consider enough qubits

0.5

In [148]:
def findNearestRational(x):
    xb = np.round(x)
    xa = x - xb
    if xa != 0:
        xaRecpb = np.round(1/xa)
        return xb*xaRecpb+1,xaRecpb
    else:
        return xb,1
      

def shor(N,a,qNum,m,psiAcc):
    HDim = 2**qNum

    K = np.gcd(a,N)
    if K != 1 and K != N:
        return N/K,K
    
    #find U_a matrix
    U_a = np.zeros([HDim,HDim],dtype=np.complex_)
    for j,k in itertools.product(range(HDim), repeat=2): 
        if j == a*k % N and k < N:
            U_a[j,k] = 1
        elif j == k and k >= N:
            U_a[j,k] = 1
            
    #random number generator models quantum probablities
    eigenVal, eigenVec = np.linalg.eig(U_a)
    
    rs = []
    for l in range(m):
        index = np.random.randint(0,high=HDim)
        phaseEst = phaseEstimation(psiAcc,U_a,eigenVec[index])
        if phaseEst != None:
            kj,rj = findNearestRational(phaseEst)
            rs.append(rj)
    
    if rs != []:
        R = np.lcm.reduce(np.array(rs,dtype=int))

        if R % 2 == 0:
            g = np.gcd(int(a**(R/2) + 1),int(N)) 
            if g != 1 and g != N:
                return N/g, g
    
    return "failed"


In [149]:
print(shor(12,5,4,6,4)) #a=5

(6.0, 2)


In [150]:
print(shor(21,5,4,6,4))

failed


In [151]:
#tested with https://blendmaster.github.io/ShorJS/
#note it doesn't work with superpositions and how the quantum randomness was simulated